In [7]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets , transforms
import matplotlib.pyplot as plt
import torch.nn as nn

In [4]:
dataset_folder = './ImgFolderDataset'


# Define transformations (resizing, converting to tensor, normalization)
transform = transforms.Compose([
    transforms.Resize((80, 80)),  # Resize images to 32x32 (or use any other size)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

In [21]:
# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root=dataset_folder, transform=transform)


In [22]:
train_data = DataLoader(dataset,batch_size=10,shuffle=True)

In [27]:
class customNN(nn.Module):
    def __init__(self,input_size, hidden_size, num_classes):
        super(customNN, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size,hidden_size)
        self.fc4 = nn.Linear(hidden_size,hidden_size)
        self.fc_out = nn.Linear(hidden_size,num_classes)

        self.relu = nn.ReLU()
    
    def forward(self,x):
        x = x.view(x.size(0),-1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc_out(x)
        return x

In [28]:
input_size = 80 * 80 * 3
hidden_size = 128
num_class = 4
model = customNN(input_size=input_size,hidden_size=hidden_size,num_classes=num_class)


In [29]:
import torch.optim as optim

# 5. Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [30]:
num_epoch = 10

for epoch in range(num_epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_data:
        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        _ , predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss/len(train_data)
    accuracy = 100 * correct/total
    print(f'Epoch [{epoch+1}/{num_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')



Epoch [1/10], Loss: 1.3899, Accuracy: 14.29%
Epoch [2/10], Loss: 1.3455, Accuracy: 52.38%
Epoch [3/10], Loss: 1.1102, Accuracy: 57.14%
Epoch [4/10], Loss: 1.2883, Accuracy: 76.19%
Epoch [5/10], Loss: 0.5034, Accuracy: 90.48%
Epoch [6/10], Loss: 0.6243, Accuracy: 95.24%
Epoch [7/10], Loss: 0.2457, Accuracy: 95.24%
Epoch [8/10], Loss: 0.1617, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0647, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0341, Accuracy: 100.00%


In [31]:
torch.save(model.state_dict(),'./nn/customdata')
print("Model saved as './nn/customdata'")

Model saved as './nn/customdata'


In [32]:
class_names = dataset.classes
class_names

['brad_pit', 'decaprio', 'johny_deep', 'keauna_reaves']